In [3]:
from coinbase.wallet.client import Client 

In [1]:
api_key = "hHlsbi4uN5ItsUV3"
secret = "mhj0iVqhvhJfoOyiTLn6IxWdDSNobcor"


In [4]:
client = Client(api_key, secret)

In [9]:
price_eth = client.get_spot_price(currency_pair= "ETH-USD")
price_btc = client.get_spot_price(currency_pair= "BTC-USD")

In [13]:
print(f'The Etherium price is ${price_eth["amount"]}, and the bitcoin price is ${price_btc["amount"]}')


The Etherium price is $1888.365, and the bitcoin price is $34940.865
